# Generating templates and recordings with MEArec

This notebook shows how to generate templates and recordings from scratch using MEArec.

The last part of the notebook showcases some plotting routines available through the package.

In [ ]:
import MEArec as mr
import MEAutility as mu
import LFPy
import yaml
from pprint import pprint
import matplotlib.pylab as åøt
%matplotlib notebook

In [2]:
LFPy.__file__

'/Users/abuccino/Documents/Codes/modeling/LFPy/LFPy/__init__.py'

## Load default configuration files

First, let's load the default configuration of MEArec

In [3]:
default_info, mearec_home = mr.get_default_config()
pprint(default_info)

{'cell_models_folder': '/Users/abuccino/.config/mearec/cell_models/bbp',
 'recordings_folder': '/Users/abuccino/Documents/Codes/mea/MEArec/data/recordings',
 'recordings_params': '/Users/abuccino/Documents/Codes/mea/MEArec/MEArec/default_params/recordings_params.yaml',
 'templates_folder': '/Users/abuccino/Documents/Codes/mea/MEArec/data/templates',
 'templates_params': '/Users/abuccino/Documents/Codes/mea/MEArec/MEArec/default_params/templates_params.yaml'}


## Generating and saving templates

In [4]:
# define cell_models folder
cell_folder = default_info['cell_models_folder']
template_params = mr.get_default_templates_params()
pprint(template_params)

{'cut_out': [2, 5],
 'delay': 10,
 'drift_steps': 30,
 'drift_xlim': [-10, 10],
 'drift_ylim': [-10, 10],
 'drift_zlim': [30, 80],
 'drifting': False,
 'dt': 0.03125,
 'max_drift': 100,
 'min_amp': 30,
 'min_drift': 30,
 'n': 50,
 'ncontacts': 10,
 'offset': 0,
 'overhang': 30,
 'probe': 'Neuronexus-32',
 'rot': 'physrot',
 'seed': None,
 'sim_time': 1,
 'target_spikes': [3, 50],
 'weights': [0.25, 1.75],
 'xlim': [10, 80],
 'ylim': None,
 'zlim': None}


Now let's change a few parameters and generate templates.

In [7]:
template_params['n'] = 10
template_params['probe'] = 'Neuronexus-32'
# the templates are not saved, but the intracellular simulations are saved in 'templates_folder'
tempgen = mr.gen_templates(cell_models_folder=cell_folder, params=template_params, n_jobs=13)
# this will take a few minutes...

Running with 13 jobs



Simulation time:  62.19468688964844 





The `tempgen` variable is a `TemplateGenerator` object. It contains the `templates`, `locations`, `rotations`, and `celltypes` of the generated templates.

In [10]:
print('Templates shape', tempgen.templates.shape)
print('Sample locations', tempgen.locations[:3])
print('Sample rotations', tempgen.rotations[:3])
print('Sample cell types', tempgen.celltypes[:3])

Templates shape (130, 32, 224)
Sample locations [[  25.59020171   34.55953062  -82.61710864]
 [  30.42507329   11.73125763 -119.49064589]
 [  23.38256901    8.33359728  -77.02424605]]
Sample rotations [[ 1.3719185  -0.02472658 -2.20004747]
 [ 1.32230512 -0.07434698 -0.99948901]
 [ 1.60932556  0.04665138  3.05103086]]
Sample cell types ['L5_BP_bAC217_1' 'L5_BP_bAC217_1' 'L5_BP_bAC217_1']


We can now save the `TemplateGenerator` object in h5 format.

In [11]:
# save templates in h5 format
mr.save_template_generator(tempgen, filename='data/test_templates.h5')


Saved  templates in data/test_templates.h5 



## Generating and saving recordings

Once the templates have been generated, we can use them to generate recordings. Let's fisrt load and take a look at the default parameters:

In [12]:
recordings_params = mr.get_default_recordings_params()
pprint(recordings_params)

{'cell_types': {'excitatory': ['PC', 'SS', 'SP'],
                'inhibitory': ['AC',
                               'BP',
                               'BC',
                               'BTC',
                               'ChC',
                               'DBC',
                               'MC',
                               'NGC']},
 'recordings': {'angle_tol': 15,
                'bursting': False,
                'chunk_duration': 0,
                'color_noise_floor': 1,
                'color_peak': 300,
                'color_q': 2,
                'drift_mode': 'slow',
                'drifting': False,
                'dtype': 'float32',
                'exp_decay': 0.2,
                'extract_waveforms': False,
                'far_neurons_exc_inh_ratio': 0.8,
                'far_neurons_max_amp': 10,
                'far_neurons_n': 300,
                'far_neurons_noise_floor': 0.5,
                'fast_drift_max_jump': 20,
                'fast_drift_m

Similarly to the templates generation, we can change th eparameters that we pass to the `gen_recordings` function.
In this case we will keep the default parameters.

In [13]:
recgen = mr.gen_recordings(templates='data/test_templates.h5', params=recordings_params)

Spiketrains seed:  9032
Noise Level  10
Templates selection seed:  5366
Selecting cells
Padding template edges
Elapsed pad time: 0.17991018295288086
Creating time jittering
Elapsed jitter time: 0.389970064163208
Computing spike train SNR
Adding spiketrain annotations
Convolution seed:  1042
Electrode modulaton
Adding noise
Noise seed:  5136
Filtering
Band-pass cutoff [ 300. 6000.] Hz
Elapsed time:  2.5588909679991048


The `recgen` variable is a `RecordingGenerator` object. It contains the selected `templates`, `template_locations`, `template_rotations`, and `template_celltypes`, as well as `spiketrains` and `recordings`.

In [14]:
print('Recordings shape', recgen.recordings.shape)
print('Selected templates shape', recgen.recordings.shape)
print('Sample template locations', recgen.template_locations[:3])
print('Number of neurons', len(recgen.spiketrains))
print('Sample spike train', recgen.spiketrains[0].times)

Recordings shape (32, 320000)
Selected templates shape (32, 320000)
Sample template locations [[ 27.45116824  40.28194951  18.14995165]
 [ 34.95372717  -3.74434058 115.69880692]
 [ 25.59778543  46.27314802  59.42980505]]
Number of neurons 10
Sample spike train [0.11429887 0.70300085 0.73083964 0.82369744 0.91580225 1.20780594
 1.26923889 1.40174683 1.80875062 1.95944768 2.12334028 2.54161796
 2.6544476  3.44370852 3.50380249 3.68940609 3.85425124 4.60363379
 4.60830634 4.66215386 5.95264095 6.20761764 6.23352204 6.25462298
 6.54975168 6.56466418 6.843358   6.93180796 7.49679611 7.52627598
 7.88818956 8.11402097 8.29930613 8.43728921 8.83530261 8.91867706
 9.19055022 9.38906899 9.53036428 9.54350781 9.55563152] s


We can now save the `RecordingGenerator` object to h5 format.

In [15]:
# save recordings in h5 format
mr.save_recording_generator(recgen, filename='data/test_recordings.h5')

## Plotting the templates and recordings

`MEArec` implements some basic plotting functions.

In [ ]:
# plot templates
mr.plot_templates(recgen, single_axes=False, ncols=4)
mr.plot_templates(recgen, single_axes=True, cmap='rainbow')

In [ ]:
mr.plot_recordings(recgen)
mr.plot_recordings(recgen, start_time=0, end_time=1, overlay_templates=True)

In [ ]:
mr.plot_waveforms(recgen, electrode='max', cmap='rainbow')

In [ ]:
mr.plot_rasters(recgen.spiketrains)

In [ ]:
_ = mr.plot_pca_map(recgen, cmap='coolwarm')